# モデルの精度を確認するためのインターフェース

ver.2022/12/19

「Train_MMVC.ipynb」で学習したモデルでTTSと非リアルタイムのVCを行い、モデルの精度を検証します。

​

In [ ]:
#@title ## 1 Google Driveをマウント
#@markdown **このノートブックで、Google Driveを使用するための設定です。**

#@markdown 「警告: このノートブックは Google が作成したものではありません。」といったポップアップが表示された場合、内容を確認して「このまま実行」を選択してください。このノートブックでは、外部へのデータ送信は一切行われません。

#@markdown 　「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されるので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

#@markdown 成功すれば、下記メッセージが出ます。

#@markdown ``` 
#@markdown Mounted at /content/drive/
#@markdown ```

from google.colab import drive
drive.mount('/content/drive')

​

In [ ]:
#@title 2 MMVC_Trainerディレクトリに移動
#@markdown ​マウントしたGoogle DriveのMMVC_Trainerディレクトリに移動します。

#@markdown Google DriveでMMVC_Trainerの場所を確認し、以下でパスを指定してください。

#@markdown 正しいパスが指定されていれば、以下のようなメッセージが表示されます。

#@markdown ```
#@markdown attentions.py
#@markdown commons.py
#@markdown ...(略)
#@markdown ```
#@markdown


#@markdown ​
#@markdown ### Settings
directory = "/content/drive/MyDrive/MMVC_Trainer" #@param {type:"string"}

%cd $directory
!ls -1

​

In [ ]:
#@title ## 3 ライブラリのインストール
#@markdown 時間がかかります。気長にお待ちください。
!apt-get install espeak
!pip install -r requirements.txt

%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

​

In [ ]:
#@title ## 4 学習したモデルの読み込み
#@markdown 学習したモデルを読み込みます。以下のSettingsで必要な設定を行った後、セルを実行してください。

#@markdown ​
#@markdown ### Settings
#@markdown CONFIG_PATH：configファイルのパス
#@markdown 作成したconfigファイル(json)を指定してください。  
#@markdown `configs/****.json` のような値になります。
CONFIG_PATH = "configs/train_config.json" #@param {type:"string"}

#@markdown NET_PATH：学習したモデルのパス
#@markdown 作成されたモデル(pth)を指定してください。
#@markdown `logs/xxxxxxxx_xxxxx/G_xxxxx.pth` のような値になります。
NET_PATH = "logs/20220306_24000/G_xxxxx.pth"  #@param {type:"string"}

hps = utils.get_hparams_from_file(CONFIG_PATH)

net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    n_speakers=hps.data.n_speakers,
    **hps.model)
_ = net_g.eval()

_ = utils.load_checkpoint(NET_PATH, net_g, None)

​

In [ ]:
#@title ## 5 学習したモデルで非リアルタイムVCを行う
#@markdown 非リアルタイムのVCを行います。以下のSettingsで必要な設定を行った後、セルを実行してください。

#@markdown 性能が悪い場合、学習不足か他に問題があります。

#@markdown ​
#@markdown ### Settings
#@markdown SOURCE_SPEAKER_ID
#@markdown ソース話者のID
SOURCE_SPEAKER_ID = 107 #@param {type:"integer"}

#@markdown SOURCE_WAVFILE
#@markdown ソース話者の音声ファイルのパス
#@markdown 学習に使用した音声ファイルか、新規に録音した音声ファイルを指定してください。
SOURCE_WAVFILE = "dataset/textful/00_myvoice/wav/VOICEACTRESS100_001.wav" #@param {type:"string"}

#@markdown TARGET_ID
#@markdown ターゲット話者のID
TARGET_ID = 100 #@param {type:"integer"}

with torch.no_grad():
    dataset = TextAudioSpeakerLoader(hps.data.validation_files_notext, hps.data)
    data = dataset.get_audio_text_speaker_pair([SOURCE_WAVFILE, SOURCE_SPEAKER_ID, "a"])
    data = TextAudioSpeakerCollate()([data])
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src = [x for x in data]
    sid_tgt1 = torch.LongTensor([TARGET_ID])
    audio1 = net_g.voice_conversion(spec, spec_lengths, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
print("Original SID: %d" % sid_src.item())
ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate))
print("Converted SID: %d" % sid_tgt1.item())
ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate))